In [1]:
# 기능적인 부분의 함수화를 중점으로 코드를 구성
# 프로젝트 내 유지보수를 높이기 위함

import folium
import pandas as pd
import heapq
import time         # 다익스트라와 플로이드-워셜로 배열을 저장한 후 사용하는 방법 중 더 효율적인 것을 찾기 위한 라이브러리
import numpy as np  # 플로이드-워셜의 결과를 저장하기 위한 라이브러리
import matplotlib.pyplot as plt
from collections import deque, defaultdict  # 최소 환승 경로를 알기 위한 라이브러리 추
plt.rcParams['font.family'] = 'Malgun Gothic'  # 또는 'NanumGothic'
plt.rcParams['axes.unicode_minus'] = False     # 마이너스 부호 깨짐 방지

In [2]:
# 데이터 읽기
df = pd.read_csv(r"C:\Users\aqtg6\Downloads\input2.csv", encoding='utf-8')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 304 entries, 0 to 303
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   line_num    304 non-null    int64  
 1   station_nm  304 non-null    object 
 2   cx          304 non-null    float64
 3   cy          304 non-null    float64
 4   link1       304 non-null    object 
 5   link2       286 non-null    object 
 6   link3       2 non-null      object 
 7   transfer1   50 non-null     object 
 8   transfer2   6 non-null      object 
dtypes: float64(2), int64(1), object(6)
memory usage: 21.5+ KB


In [3]:
# df에 있는 데이터를 통해 graph와 node, edge의 그래프를 구축
# 파싱, 정제, 그래프화의 기능을 모두 포함하므로 preprocessing으로 이름 지음
# 시각화를 위해 역 이름을 key값으로 (cx, cy) 튜플을 값으로 저장하는 딕셔너리도 생성

def preprocessing(row, col_names, graph, nodes, edges, station_coords):
    station_nm = str(row['station_nm']).strip()                    # 공백, 개행, 타입 불일치 등 예방
    line_num = str(row['line_num']).strip()                        # 공백, 개행, 타입 불일치 등 예방
    from_node = f"{row['station_nm']}역_{row['line_num']}"
    cx = float(row['cx'])
    cy = float(row['cy'])
    station_coords[from_node] = (cy, cx)
    nodes.add(from_node)
    for col in col_names:
        val = row.get(col)
        if pd.notna(val) and val:
            items = val.split()
            for item in items:
                left, travel_time = item.split('):')
                left = left.strip('(')
                to_station, to_line = left.split(',')
                to_node = f"{to_station}역_{to_line}"
                nodes.add(to_node)
                if from_node not in graph:
                    graph[from_node] = []
                graph[from_node].append((to_node, float(travel_time)))
                edges.append((from_node, to_node, float(travel_time)))


In [4]:
# preprocessing 함수를 실행하는 코드 블럭

graph = {}          # 인접 리스트 구현을 위해서 dict 사용
nodes = set()       # 중복 허용을 막기 위해서 
edges = []          # 간선 담을 리스트
station_coords = {} # 시각화 할 상대좌표 받기

link_cols = [f'link{i}' for i in range(1, 6)]
transfer_cols = [f'transfer{i}' for i in range(1, 4)]

for idx, row in df.iterrows():
    preprocessing(row, link_cols, graph, nodes, edges, station_coords)
    preprocessing(row, transfer_cols, graph, nodes, edges, station_coords)


# 리스트로 변환
nodes = list(nodes)   # 다루기 쉽게 리스트로 변환

In [5]:
# 직접 실제 지도에 노선 그려서 시각화

def plot_path_folium(path, station_coords, zoom_start=13):
    line_colors = {"1": "blue", "2": "green", "3": "orange", "4": "#00a2e8", "5": "purple"} # #00a2e8은 css 색상 코드임 : folium에서 지원하는 lightblue가 너무 연함
    latlons = [station_coords[station] for station in path if station in station_coords]
    start_lat, start_lon = latlons[0]
    m = folium.Map(location=[start_lat, start_lon], zoom_start=zoom_start)

    # 역마다 마커 및 CircleMarker(호선 색상) 추가
    for station in path:
        if station in station_coords:
            lat, lon = station_coords[station]
            _, line = station.split("_", 1)
            color = line_colors.get(line, "blue")
            folium.Marker(
                [lat, lon],
                popup=station,
                tooltip=station,
                icon=folium.Icon(color='red' if station == path[0] or station == path[-1] else 'blue')
            ).add_to(m)
            folium.CircleMarker(
                location=[lat, lon],
                radius=5,
                color=color,
                fill=True,
                fill_color=color,
                fill_opacity=0.7
            ).add_to(m)

    # 호선이 바뀌는 구간마다 PolyLine을 따로 그림
    sub_path = [latlons[0]]
    prev_line = path[0].split("_", 1)[1]
    for idx, station in enumerate(path[1:], 1):
        if station in station_coords:
            curr_line = station.split("_", 1)[1]
            if curr_line != prev_line:
                folium.PolyLine(sub_path, color=line_colors.get(prev_line, "blue"), weight=5, opacity=0.7).add_to(m)
                sub_path = []
            sub_path.append(station_coords[station])
            prev_line = curr_line
    # 마지막 구간 PolyLine 그리기
    if sub_path:
        folium.PolyLine(sub_path, color=line_colors.get(prev_line, "blue"), weight=5, opacity=0.7).add_to(m)

    return m

In [6]:
# 사용자로부터 입력받는 부분
# 사용자가 잘못된 입력을 할 수 있으므로 예외처리

def get_valid_node(prompt, nodes):
    while True:
        try:
            print(prompt)
            name, num = input().split()
            node = f"{str(name).strip()}_{str(num).strip()}"       # 공백, 개행, 타입 불일치 등 예방
            if node not in nodes:
                print(f"'{name} {num}'은(는) 존재하지 않는 역/호선입니다. 다시 입력해주세요.")
                continue
            return node
        except ValueError:
            print("입력 형식이 잘못되었습니다. 예시처럼 '서울역 1'과 같이 입력해주세요.")

In [7]:
# 경로 복원하기 위한 함수

def reconstruct_path(start, end, path_tracker, nodes=None, node_indices=None):
    if isinstance(path_tracker, dict):
        # 다익스트라용 (prev_route 딕셔너리)
        path = []
        node = end
        while node is not None:
            path.append(node)
            node = path_tracker[node]
        path.reverse()
        return path
    else:
        # 플로이드-워셜용 (next_node 2D 리스트)
        i = node_indices[start]
        j = node_indices[end]
        if path_tracker[i][j] is None:
            return []
        path = [start]
        while i != j:
            i = path_tracker[i][j]
            path.append(nodes[i])
        return path


In [8]:
# y환승 구역을 처리하는 코드블럭

# Y환승 처리
y_transfer = {"금천구청광명역":"금천구청역", "병점서동탄역":"병점역", "강동길동역":"강동역", "구로구일역":"구로역"}

# Y환승 역 탐지
y_station = {"금천구청광명역":5, "병점서동탄역":5, "강동길동역":1, "구로구일역":2}
y_station2 = {"금천구청역":5, "병점역":5, "강동역":1, "구로역":2, "광명역":5, "서동탄":5.1, "길동":1.6, "구일":2}

# Y환승 탐지하는지 확인
y_detection = {"금천구청광명역":("광명역", "석수역"), "병점서동탄역":("서동탄역", "세마역"), "강동길동역":("길동역", "둔촌동역"), "구로구일역":("구일역", "가산디지털단지역")}

def y_clear(path, graph):
    if not path or len(path) < 3:
        return path, 0
    
    optimized_path = path.copy()
    saved_time = 0
    
    # 역순으로 처리하여 인덱스 변화 문제 방지
    for i in range(len(path) - 1, 0, -1):
        current_station = path[i].rsplit('_', 1)[0]
        
        # Y환승 키값이 경로에 있는지 확인
        if current_station in y_transfer:
            # 경로의 시작이나 끝이면 건너뛰기
            if i == 0 or i == len(path) - 1:
                continue
            
            # 이전 역과 다음 역 확인
            prev_station = path[i-1].rsplit('_', 1)[0]
            if i+2 < len(path):
                next_station = path[i+2].rsplit('_', 1)[0]
            else:
                # 경로가 충분하지 않으면 일반적인 다음 역 확인
                if i+1 < len(path):
                    next_station = path[i+1].rsplit('_', 1)[0]
                else:
                    continue
            
            # Y환승 감지 조건 확인
            detection_key = current_station
            if detection_key in y_detection:
                chk_1, chk_2 = y_detection[detection_key]
                # 실제 Y환승이 발생하지 않는 경우
                if not ((prev_station == chk_1 and next_station == chk_2) or (prev_station == chk_2 and next_station == chk_1)):
                    
                    # Y환승 중간역 제거 및 시간 계산
                    removed_time = 0
                    
                    # 이전 역 -> Y환승역 시간
                    for neighbor, weight in graph.get(path[i-1], []):
                        if neighbor == path[i]:
                            removed_time += weight
                            break
                    
                    # Y환승역 -> 다음 역 시간
                    for neighbor, weight in graph.get(path[i], []):
                        if neighbor == path[i+1]:
                            removed_time += weight
                            break
                    
                    # 직접 연결 시간 확인 (이전 역 -> 다음 역)
                    direct_time = y_station[current_station]
                            
                    saved_time += (removed_time - direct_time)
                    optimized_path.pop(i)
    
    return optimized_path, saved_time

In [9]:
def path_print(path, graph):
    transfer_count = 0
    if not path or len(path) < 2:
        print("경로가 충분하지 않습니다.")
        return

    prev_line = path[0].rsplit('_', 1)[1]
    prev_station = path[0].rsplit('_', 1)[0]
    
    # Y환승 역명 정규화
    if prev_station in y_transfer:
        prev_station = y_transfer[prev_station]
        
    stations = [f"{prev_station}({prev_line}호선)"]
    time_sum = 0
    
    for i in range(1, len(path)):
        cur_station, cur_line = path[i].rsplit('_', 1)
        prev_node = path[i-1]
        
        # graph에서 시간 조회
        time = None
        for neighbor, weight in graph.get(prev_node, []):
            if neighbor == path[i]:
                time = weight
                break
        if time == None:
            time = y_station2[cur_station]

        time_sum += time

        # Y환승 역명 정규화
        original_cur_station = cur_station
        if cur_station in y_transfer:
            cur_station = y_transfer[cur_station]

        # 환승 조건 체크
        should_transfer = False
        transfer_type = ""
        
        if cur_line != prev_line:
            # 호선 변경 환승
            should_transfer = True
            transfer_type = f"[{prev_line}호선에서 {cur_line}호선으로 환승]"

        elif cur_station == prev_station:
            should_transfer = True
            transfer_type = f"[{cur_station} 내 분기 환승]"


        if should_transfer:
            print(f"{format_time(time_sum - time)}")
            print(" → ".join(stations))
            print(f"{transfer_type} : {format_time(time)}")
            print("------------------------------------------")
            stations = []
            time_sum = 0
            transfer_count += 1
            
        stations.append(f"{cur_station}({cur_line}호선)")
        prev_line = cur_line
        prev_station = cur_station
        
    if stations:
        print(f"{format_time(time_sum)}")
        print(" → ".join(stations))
        print("-------------------도착-------------------")
        print(f"환승 횟수 : {transfer_count}")

In [10]:
# 소요 시간을 시간, 분, 초 단위로 환산 후 출력

def format_time(total_time):
    hours = int(total_time // 60)
    minutes = int(total_time % 60)
    seconds = int(round((total_time - int(total_time)) * 60))

    # 초가 60이 되는 경우(반올림 등) 보정
    if seconds == 60:
        seconds = 0
        minutes += 1
    if minutes == 60:
        minutes = 0
        hours += 1

    result = []
    if hours > 0:
        result.append(f"{hours}시간")
    if minutes > 0:
        result.append(f"{minutes}분")
    if seconds > 0:
        result.append(f"{seconds}초")

    # 만약 모두 0이면 0초 출력
    if not result:
        return "0초"
    return " ".join(result)


In [11]:
# 최소 환승

def min_transfer_and_time(start_node, end_node, graph, nodes):
    # 1. 환승 정보 구축 (같은 역, 다른 호선)
    station_to_nodes = defaultdict(list)
    for node in nodes:
        station, line = node.rsplit('_', 1)
        station_to_nodes[station].append(node)

    # 2. 우선순위 큐 초기화(환승 횟수, 걸린 시간, 역 이름_호선, 호선, 경로)
    heap = []
    start_station, start_line = start_node.rsplit('_', 1)
    end_station, end_line = end_node.rsplit('_', 1)
    for node in station_to_nodes[start_station]:
        line = node.rsplit('_', 1)[1]
        if line == start_line:
            heapq.heappush(heap, (0, 0, node, line, [node]))
        else:
            # graph에서 start_node와 node(환승노드) 사이의 환승시간을 찾아서 넣기
            transfer_time = None
            for neighbor, weight in graph.get(start_node, []):
                if neighbor == node:
                    transfer_time = weight
                    break
            if transfer_time is not None:
                heapq.heappush(heap, (1, transfer_time, node, line, [node]))
            # 만약 환승 간선이 없다면 추가하지 않음

    # 3. 방문 체크
    visited = dict()

    while heap:
        transfer_cnt, total_time, current, cur_line, path = heapq.heappop(heap)

        # 현재가 도착역이면
        if current == end_node:
            return transfer_cnt, total_time, path

        key = (current, cur_line)
        if key in visited:
            prev_transfer, prev_time = visited[key]
            if (transfer_cnt, total_time) >= (prev_transfer, prev_time): # 파이썬에서 튜플끼리의 비교는 사전식 비교
                continue
                
        visited[key] = (transfer_cnt, total_time)

        for neighbor, weight in graph.get(current, []):
            neighbor_line = neighbor.rsplit('_', 1)[1]
            if neighbor_line == cur_line:
                # 같은 호선 이동
                heapq.heappush(heap, (transfer_cnt, total_time + weight, neighbor, cur_line, path + [neighbor]))
            else:
                # 환승
                heapq.heappush(heap, (transfer_cnt + 1, total_time + weight, neighbor, neighbor_line, path + [neighbor]))


    return -1, -1, []


In [12]:
# 다익스트라
# O(E + VlogV) (E: 간선 수, V: 노드 수)
# 쿼리가 적을 때 유리 + 메모리 사용량이 적고 사전 계산이 필요 없음

def dijkstra(graph, start, end):
    # (누적 시간, 노드) 형태로 저장
    heap = []
    heapq.heappush(heap, (0, start))

    distances = {node: float('inf') for node in graph}
    distances[start] = 0

    prev_route = {node: None for node in graph}          # 경로 복원을 위한 사전

    while heap:
        cost, current_node = heapq.heappop(heap)

        if cost > distances[current_node]:
            continue
        next_step = graph[current_node]
        for next_node, weight in graph[current_node]:
            next_distance = cost + weight
            if next_distance < distances[next_node]:
                distances[next_node] = next_distance
                prev_route[next_node] = current_node
                heapq.heappush(heap, (next_distance, next_node))

    return distances[end], prev_route

In [13]:
# 플로이드-워셜
# 실행 할 때마다 다익스트라 알고리즘을 계속 수행해야 함
# 플로이드-워셜 알고리즘을 통해 미리 최단거리와 최단 경로를 파일로 저장한 후
# 새롭게 사용할 때마다 파일을 불러와서 사용하는 방식으로 구현
# 처음에만 O(V^3)이지만 이후의 쿼리에는 배열 인덱싱만 하면 되므로 O(1) (매우 빠름)
# 메모리 사용량이 증가하지만 속도가 빠름(쿼리가 많을수록 유리)

def floyd_warshall(nodes, edges):
    node_indices = {node: idx for idx, node in enumerate(nodes)}
    N = len(nodes)
    INF = float('inf')
    dist = [[INF] * N for _ in range(N)]
    next_node = [[None] * N for _ in range(N)]

    for i in range(N):
        dist[i][i] = 0

    for from_node, to_node, weight in edges:
        i, j = node_indices[from_node], node_indices[to_node]
        dist[i][j] = weight
        next_node[i][j] = j

    for k in range(N):
        for i in range(N):
            for j in range(N):
                if dist[i][j] > dist[i][k] + dist[k][j]:
                    dist[i][j] = dist[i][k] + dist[k][j]
                    next_node[i][j] = next_node[i][k]
    
    return dist, next_node, node_indices

In [14]:
start_node = get_valid_node("출발 역과 호선을 입력해주세요 (예: 서울역 1): ", nodes)
end_node = get_valid_node("도착 역과 호선을 입력해주세요 (예: 강남역 2): ", nodes)

출발 역과 호선을 입력해주세요 (예: 서울역 1): 


 서울역 1


도착 역과 호선을 입력해주세요 (예: 강남역 2): 


 강남역 2


In [16]:
# 실제 실행 후 결과를 출력하는 스크립트
# time.time()은 초 단위(소수점 6자리 정도)로 측정되기 때문에 수 ms 이하는 부정확
# time.perf_counter()는 보다 정밀하기 ns까지 측정 가능

start_time = time.perf_counter()                                        # 시간 측정 시작
total_time, prev_route = dijkstra(graph, start_node, end_node)
end_time = time.perf_counter()                                          # 시간 측정 종료
path = reconstruct_path(start_node, end_node, prev_route)

optimized_path1, saved_time = y_clear(path, graph)
print("-----------------최단경로-----------------")
print("총 소요 시간:", format_time(total_time - saved_time))
print("-------------------출발-------------------")
path_print(optimized_path1, graph)
print()


min_transfers, total_time, transfer_path = min_transfer_and_time(start_node, end_node, graph, nodes)
optimized_path2, saved_time = y_clear(transfer_path, graph)
print("-----------------최소환승-----------------")
print(f"최소 환승 경로 소요 시간: {format_time(total_time-saved_time)}")
print("-------------------출발-------------------")
path_print(optimized_path2, graph)
print()

m = plot_path_folium(optimized_path1, station_coords)
m

-----------------최단경로-----------------
총 소요 시간: 25분
-------------------출발-------------------
0초
서울역(1호선)
[1호선에서 4호선으로 환승] : 2분 12초
------------------------------------------
13분
서울역(4호선) → 숙대입구역(4호선) → 삼각지역(4호선) → 신용산역(4호선) → 이촌역(4호선) → 동작역(4호선) → 총신대입구역(4호선) → 사당역(4호선)
[4호선에서 2호선으로 환승] : 1분
------------------------------------------
8분 48초
사당역(2호선) → 방배역(2호선) → 서초역(2호선) → 교대역(2호선) → 강남역(2호선)
-------------------도착-------------------
환승 횟수 : 2

-----------------최소환승-----------------
최소 환승 경로 소요 시간: 40분 24초
-------------------출발-------------------
2분
서울역(1호선) → 시청역(1호선)
[1호선에서 2호선으로 환승] : 1분 24초
------------------------------------------
37분
시청역(2호선) → 을지로입구역(2호선) → 을지로3가역(2호선) → 을지로4가역(2호선) → 동대문역사문화공원역(2호선) → 신당역(2호선) → 상왕십리역(2호선) → 왕십리역(2호선) → 한양대역(2호선) → 뚝섬역(2호선) → 성수역(2호선) → 건대입구역(2호선) → 구의역(2호선) → 강변역(2호선) → 잠실나루역(2호선) → 잠실역(2호선) → 잠실새내역(2호선) → 종합운동장역(2호선) → 삼성역(2호선) → 선릉역(2호선) → 역삼역(2호선) → 강남역(2호선)
-------------------도착-------------------
환승 횟수 : 1



In [18]:
# 다익스트라 실행 소요 시간

dijkstra_time = end_time - start_time
print(f"다익스트라 실행 시간: {dijkstra_time:.10f}초")

다익스트라 실행 시간: 0.0010694000초


In [21]:
# 플로이드-워셜 실행 및 결과 저장

start_time = time.perf_counter()
dist, next_node, node_indices = floyd_warshall(nodes, edges)
end_time = time.perf_counter()
total_time = dist[node_indices[start_node]][node_indices[end_node]]
path = reconstruct_path(start_node, end_node, next_node, nodes, node_indices)

optimized_path1, saved_time = y_clear(path, graph)
print("-----------------최단경로-----------------")
print("총 소요 시간:", format_time(total_time - saved_time))
print("-------------------출발-------------------")
path_print(optimized_path1, graph)
print()

min_transfers, total_time, transfer_path = min_transfer_and_time(start_node, end_node, graph, nodes)
optimized_path2, saved_time = y_clear(transfer_path, graph)
print("-----------------최소환승-----------------")
print(f"최소 환승 경로 소요 시간: {format_time(total_time-saved_time)}")
print("-------------------출발-------------------")
path_print(optimized_path2, graph)
print()

m = plot_path_folium(optimized_path1, station_coords)
m

-----------------최단경로-----------------
총 소요 시간: 25분
-------------------출발-------------------
0초
서울역(1호선)
[1호선에서 4호선으로 환승] : 2분 12초
------------------------------------------
13분
서울역(4호선) → 숙대입구역(4호선) → 삼각지역(4호선) → 신용산역(4호선) → 이촌역(4호선) → 동작역(4호선) → 총신대입구역(4호선) → 사당역(4호선)
[4호선에서 2호선으로 환승] : 1분
------------------------------------------
8분 48초
사당역(2호선) → 방배역(2호선) → 서초역(2호선) → 교대역(2호선) → 강남역(2호선)
-------------------도착-------------------
환승 횟수 : 2

-----------------최소환승-----------------
최소 환승 경로 소요 시간: 40분 24초
-------------------출발-------------------
2분
서울역(1호선) → 시청역(1호선)
[1호선에서 2호선으로 환승] : 1분 24초
------------------------------------------
37분
시청역(2호선) → 을지로입구역(2호선) → 을지로3가역(2호선) → 을지로4가역(2호선) → 동대문역사문화공원역(2호선) → 신당역(2호선) → 상왕십리역(2호선) → 왕십리역(2호선) → 한양대역(2호선) → 뚝섬역(2호선) → 성수역(2호선) → 건대입구역(2호선) → 구의역(2호선) → 강변역(2호선) → 잠실나루역(2호선) → 잠실역(2호선) → 잠실새내역(2호선) → 종합운동장역(2호선) → 삼성역(2호선) → 선릉역(2호선) → 역삼역(2호선) → 강남역(2호선)
-------------------도착-------------------
환승 횟수 : 1



In [22]:
# 플로이드-워셜 실행 시간

print(f"플로이드-워셜 실행 시간: {end_time - start_time:.10f}초")

플로이드-워셜 실행 시간: 8.4736652000초


In [29]:
# 플로이드 워셜의 결과를 저장해두고 나중에 최단 경로를 필요로 할 때 load해서 사용
# 저장하는 코드

next_node_array = np.array(next_node, dtype=object)   # next_node에 None 값이 있을 수 있기 때문에 object로 형변환
dist_array = np.array(dist)

np.savez('floyd_warshall_save.npz', next_node=next_node_array, dist=dist_array)

In [33]:
# 한 번 계속 불러와서 재사용 하는 코드

recall_start = time.perf_counter()                            # 밑준비 시간 (파일 불러오기 + 인덱싱)
data = np.load('floyd_warshall_save.npz', allow_pickle=True)
next_node = data['next_node']
dist = data['dist']
node_indices = {node: idx for idx, node in enumerate(nodes)}       
recall_end = time.perf_counter()

recall_time = recall_end - recall_start
print(f"파일 부르고 인덱싱 시간: {recall_time:.10f}초")

파일 부르고 인덱싱 시간: 0.0123918000초


In [34]:
# 재사용 할 시 사용하는 코드

def get_shortest_distance(start, end, node_indices, dist):
    i, j = node_indices[start], node_indices[end]
    return dist[i][j]

In [35]:
# 지속적인 쿼리 시의 사용하는 입력받을 코드 블럭

start_node = get_valid_node("출발 역과 호선을 입력해주세요 (예: 서울역 1): ", nodes)
end_node = get_valid_node("도착 역과 호선을 입력해주세요 (예: 강남역 2): ", nodes)

출발 역과 호선을 입력해주세요 (예: 서울역 1): 


 서울역 1


도착 역과 호선을 입력해주세요 (예: 강남역 2): 


 강남역 2


In [42]:
# 비교를 위한 시간 처리
# 최단 경로와 시간 계산

reuse_start_time = time.perf_counter()                                             # 시간 측정 시작
shortest_distance = get_shortest_distance(start_node, end_node, node_indices, dist)
reuse_end_time = time.perf_counter()                                               # 시간 측정 종료

path = reconstruct_path(start_node, end_node, next_node, nodes, node_indices)
total_time = dist[node_indices[start_node]][node_indices[end_node]]

In [43]:
optimized_path1, saved_time = y_clear(path, graph)
print("-----------------최단경로-----------------")
print("총 소요 시간:", format_time(total_time - saved_time))
print("-------------------출발-------------------")
path_print(optimized_path1, graph)
print()

min_transfers, total_time, transfer_path = min_transfer_and_time(start_node, end_node, graph, nodes)
optimized_path2, saved_time = y_clear(transfer_path, graph)
print("-----------------최소환승-----------------")
print(f"최소 환승 경로 소요 시간: {format_time(total_time-saved_time)}")
print("-------------------출발-------------------")
path_print(optimized_path2, graph)
print()

m = plot_path_folium(optimized_path1, station_coords)
m

-----------------최단경로-----------------
총 소요 시간: 25분
-------------------출발-------------------
0초
서울역(1호선)
[1호선에서 4호선으로 환승] : 2분 12초
------------------------------------------
13분
서울역(4호선) → 숙대입구역(4호선) → 삼각지역(4호선) → 신용산역(4호선) → 이촌역(4호선) → 동작역(4호선) → 총신대입구역(4호선) → 사당역(4호선)
[4호선에서 2호선으로 환승] : 1분
------------------------------------------
8분 48초
사당역(2호선) → 방배역(2호선) → 서초역(2호선) → 교대역(2호선) → 강남역(2호선)
-------------------도착-------------------
환승 횟수 : 2

-----------------최소환승-----------------
최소 환승 경로 소요 시간: 40분 24초
-------------------출발-------------------
2분
서울역(1호선) → 시청역(1호선)
[1호선에서 2호선으로 환승] : 1분 24초
------------------------------------------
37분
시청역(2호선) → 을지로입구역(2호선) → 을지로3가역(2호선) → 을지로4가역(2호선) → 동대문역사문화공원역(2호선) → 신당역(2호선) → 상왕십리역(2호선) → 왕십리역(2호선) → 한양대역(2호선) → 뚝섬역(2호선) → 성수역(2호선) → 건대입구역(2호선) → 구의역(2호선) → 강변역(2호선) → 잠실나루역(2호선) → 잠실역(2호선) → 잠실새내역(2호선) → 종합운동장역(2호선) → 삼성역(2호선) → 선릉역(2호선) → 역삼역(2호선) → 강남역(2호선)
-------------------도착-------------------
환승 횟수 : 1



In [44]:
reuse_spend_time = reuse_end_time - reuse_start_time
print(f"배열 재사용 실행 시간: {reuse_spend_time:.10f}초")

배열 재사용 실행 시간: 0.0001212000초


# 결론

# 다익스트라 1번 = 0.001s
# 플로이드-워셜 1번 = 8.5s
# 파일 불러오는 시간 = 0.01s
# 불러온 파일을 기반으로 최단거리를 계산하는 시간 = 0.0001s